In [4]:
!pip install scikit-learn
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.5 MB/s eta 0:00:0000:0100:01m
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 2.8 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


2025-02-24 11:20:15.504599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Define paths and categories
train_path = "Dataset/train"
categories = [
    "Acne and Rosacea Photos",
    "Eczema Photos",
    "Hair Loss Photos Alopecia and other Hair Diseases",
    "Nail Fungus and other Nail Disease",
    "Scabies Lyme Disease and other Infestations and Bites",
]

In [6]:
# Load images and labels
images = []
labels = []

for idx, category in enumerate(categories):
    category_path = os.path.join(train_path, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            img = cv2.resize(img, (180, 180))  # Resize to match VGG19 input size
            images.append(img)
            labels.append(idx)

In [7]:
# Convert to numpy arrays
images = np.array(images) / 255.0  # Normalize pixel values
labels = np.array(labels)

In [8]:
# One-hot encode labels
num_classes = len(categories)
labels = to_categorical(labels, num_classes=num_classes)

In [9]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [10]:
# Load VGG19 model for feature extraction
vgg_model = VGG19(weights="imagenet", include_top=False, input_shape=(180, 180, 3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [11]:
# Freeze all layers in VGG19
for layer in vgg_model.layers:
    layer.trainable = False

In [12]:
# Extract features
features_train = vgg_model.predict(x_train)
features_test = vgg_model.predict(x_test)

95/95 ━━━━━━━━━━━━━━━━━━━━ 426s 4s/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 124s 5s/step


In [13]:
# Reshape features
x_train_features = features_train.reshape(features_train.shape[0], -1)
x_test_features = features_test.reshape(features_test.shape[0], -1)

In [14]:
# Define a simple dense model
model = Sequential([
    Dense(200, activation="relu"),
    Dense(170, activation="relu"),
    Dense(num_classes, activation="softmax")
])

In [15]:
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [84]:
# Train the model
history = model.fit(
    x_train_features, y_train,
    validation_data=(x_test_features, y_test),
    epochs=100,
    batch_size=32
)

Epoch 1/100
95/95 [==============================] - 1s 7ms/step - loss: 1.1907 - accuracy: 0.5624 - val_loss: 0.9723 - val_accuracy: 0.6011
Epoch 2/100
95/95 [==============================] - 0s 3ms/step - loss: 0.8178 - accuracy: 0.6889 - val_loss: 0.7911 - val_accuracy: 0.7107
Epoch 3/100
95/95 [==============================] - 0s 3ms/step - loss: 0.7068 - accuracy: 0.7279 - val_loss: 0.8196 - val_accuracy: 0.6922
Epoch 4/100
95/95 [==============================] - 0s 3ms/step - loss: 0.5705 - accuracy: 0.7910 - val_loss: 0.7856 - val_accuracy: 0.7252
Epoch 5/100
95/95 [==============================] - 0s 3ms/step - loss: 0.5252 - accuracy: 0.8015 - val_loss: 0.7293 - val_accuracy: 0.7252
Epoch 6/100
95/95 [==============================] - 0s 3ms/step - loss: 0.4320 - accuracy: 0.8415 - val_loss: 1.0648 - val_accuracy: 0.6314
Epoch 7/100
95/95 [==============================] - 0s 4ms/step - loss: 0.4198 - accuracy: 0.8345 - val_loss: 0.7694 - val_accuracy: 0.7411
Epoch 8/100
9

In [85]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test_features, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

24/24 [==============================] - 0s 5ms/step - loss: 1.8344 - accuracy: 0.7530
Test Accuracy: 75.30%


In [86]:
model.save("skin_disease_model.h5")

In [87]:
# Function to predict skin disease
def predict_skin_disease(image_path):
    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Extract features using VGG19
    img_features = vgg_model.predict(img)
    img_features = img_features.reshape(1, -1)

    # Make prediction
    pred = model.predict(img_features)[0]
    predicted_class = np.argmax(pred)
    return categories[predicted_class]

In [92]:
# Test the function
test_image_path = "Dataset/test/Acne and Rosacea Photos/07SteroidPerioral.jpg"
predicted_disease = predict_skin_disease(test_image_path)
print(f"Predicted Disease: {predicted_disease}")

1/1 [==============================] - 0s 33ms/step
Predicted Disease: Acne and Rosacea Photos
